In [1]:
import requests
import json
import pandas as pd
import time
from dotenv import load_dotenv
import os
import re
from datetime import datetime
from decimal import Decimal, InvalidOperation
import base64
from datetime import datetime, timedelta
import pytz

pd.options.display.float_format = '{:,.14f}'.format

C:\Users\victo\AppData\Local\Temp\ipykernel_28044\815554912.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [15]:
# setup
load_dotenv()
TOKEN_URL = 'https://api.ticket360.com.br/auth/oauth/access_token'
API_URL = 'https://api.ticket360.com.br'
EVENT_ID = '30617'
EVENT_NAME = 'Camarote-essepe-2026-grupo-especial-thiaguinho'
MAX_RETRIES = 3  #Max request until fail
TIMEOUT = 30  # Seconds
DATA_DIR = 'ticket_data'
CONSOLIDATED_FILE = os.path.join(DATA_DIR, "consolidated.json")

os.makedirs(DATA_DIR,exist_ok=True)

In [16]:
def get_access_token():
    '''Get access token with error treatment'''
    for attempt in range(MAX_RETRIES):
        try:
            auth_string = f"{os.getenv('CLIENT_ID')}:{os.getenv('CLIENT_SECRET')}"
            auth_base64 = base64.b64encode(auth_string.encode()).decode()
            
            response = requests.post(
                TOKEN_URL,
                headers={
                    "Authorization": f"Basic {auth_base64}",
                    "Content-Type": "application/x-www-form-urlencoded"
                },
                data={"grant_type": "client_credentials"},
                timeout=TIMEOUT
            )
            response.raise_for_status()
            return response.json().get("access_token")
        except requests.exceptions.Timeout:
            print(f"Timeout (tentativa {attempt + 1}/{MAX_RETRIES})")
            if attempt < MAX_RETRIES - 1:
                time.sleep(5)
        except Exception as e:
            print(f"Error to get token: {type(e).__name__} - {str(e)}")
            return None
    return None

In [17]:
def fetch_report(token, start_date=None, end_date=None):
    '''Search data from API using pagination'''
    try:
        base_url = f"{API_URL}/sales/reports/consolidated/{EVENT_ID}?filter=status=paid&ticket.status=active"
        
        if start_date:
            base_url += f"&startDate={start_date}"
        if end_date:
            base_url += f"&endDate={end_date}"
        
        offset = 0
        limit = 1000
        all_sales = []
        
        while True:
            url = f"{base_url}&limit={limit}&offset={offset}"
            response = requests.get(
                url,
                headers={"Authorization": f"Bearer {token}"},
                timeout=TIMEOUT
            )
            response.raise_for_status()
            data = response.json()
            sales = data.get('sales', [])
            all_sales.extend(sales)
            
            if len(sales) < limit:
                break
                
            offset += limit
        
        return {'sales': all_sales}
    except Exception as e:
        print(f"Error to search data: {type(e).__name__} - {str(e)}")
        return None

In [18]:
def remove_today_data(df):
    hoje = datetime.now().date()

    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce", utc=True)
        df["date_only"] = df["date"].dt.date
        df = df[df["date_only"] != hoje].drop(columns=["date_only"])
    else:
        print("'Date' column not found")

    return df 

In [19]:
def save_to_json(data, filename):
    """Salva DataFrame ou dict em JSON, garantindo serialização sem referências circulares"""
    def convert(obj):
        if isinstance(obj, (pd.Timestamp, datetime)):
            return obj.isoformat()
        if isinstance(obj, (pd.Series, pd.Index, pd.Categorical)):
            return obj.tolist()
        if isinstance(obj, (set,)):
            return list(obj)
        return str(obj)  # fallback para tipos inesperados

    with open(filename, "w", encoding="utf-8") as f:
        if isinstance(data, pd.DataFrame):
            json_data = data.to_dict(orient="records")
            json.dump(json_data, f, indent=2, ensure_ascii=False, default=convert)
        else:
            json.dump(data, f, indent=2, ensure_ascii=False, default=convert)

In [20]:
def normalize_dates(df,date_column="date"):
    if date_column not in df.columns:
        return df
    
    df[date_column] = pd.to_datetime(df[date_column], utc=True, errors="coerce")
    df[date_column] = df[date_column].dt.strftime("%Y-%m-%dT%H:%M:%S%z")
    df[date_column] = df[date_column].str.replace(r"(\+)(\d{2})(\d{2})$", r"\1\2:\3", regex=True)
    return df


In [21]:
def consolidate_data(new_data_file):
    '''Faz append com novos dados mantendo formato array JSON'''
    # Carregar dados existentes
    if os.path.exists(CONSOLIDATED_FILE):
        try:
            with open(CONSOLIDATED_FILE,"r", enconding="utf-8") as f:
                data_existente = json.load(f)
            df_existente = pd.DataFrame(data_existente)
        except Exception:
            df_existente = pd.DataFrame()
    else:
        df_existente = pd.DataFrame()
    
    # Carregar novos dados
    with open(new_data_file,"r",encoding="utf-8") as f:
        data_novos = json.load(f)
    df_novos = pd.DataFrame(data_novos)
    
    # Combinar dados
    if df_existente.empty:
        df_final = df_novos
    else:
        df_final = pd.concat([df_existente, df_novos], ignore_index=True)
    
    with open(CONSOLIDATED_FILE,"w",encoding="utf-8") as f:
        json.dump(
            df_final.to_dict(orient="records"),
            f,
            indent=2,
            ensure_ascii=False,
            default=str
        )
    return df_final

In [22]:
def main():
    print("Starting data collecting...")
    token = get_access_token()
    if not token:
        print("Fail at authentication")
        return
        
    # Coletar todos os dados
    report_data = fetch_report(token)
    if not report_data or 'sales' not in report_data:
        print("Fail at get historical data.")
        return
    
    # Converter para DataFrame
    df = pd.DataFrame(report_data['sales'])

    #Função para normalizar os dados em 'date'
    df = normalize_dates(df, date_column="date")
    
    
    # Remover dados do dia atual
    df = remove_today_data(df)
    
    
    # Salvar dados processados
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    processed_file = os.path.join(DATA_DIR, f"processed_historical_{timestamp}_{EVENT_NAME}.json")
    save_to_json(df, processed_file)
    print(f"Dados históricos processados salvos em: {processed_file}")
    

    # Consolidar dados
    df_consolidated = consolidate_data(processed_file)
    print(f"Dados consolidados salvos em: {CONSOLIDATED_FILE} ({len(df_consolidated)} registros)")


if __name__ == "__main__":
    main()

Starting data collecting...
Dados históricos processados salvos em: ticket_data\processed_historical_20250903_200958_Camarote-essepe-2026-grupo-especial-thiaguinho.json
Dados consolidados salvos em: ticket_data\consolidated.json (782 registros)


In [2]:

def mostrar_datas_extremas(caminho_arquivo):
    # Abrir e ler o arquivo JSON
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        dados = json.load(f)

    # Extrair as datas (assumindo que cada item tem a chave 'date')
    lista_datas = [item['date'] for item in dados]

    # Converter as strings para objetos datetime
    lista_datas_convertidas = [datetime.fromisoformat(d) for d in lista_datas]

    # Encontrar a mais antiga e a mais recente
    data_mais_antiga = min(lista_datas_convertidas)
    data_mais_recente = max(lista_datas_convertidas)

    # Mostrar no formato ISO (AAAA-MM-DD)
    print("Data mais antiga:", data_mais_antiga.date())
    print("Data mais recente:", data_mais_recente.date())

    #Transformar o arquivo JSON em um dataframe
    df = pd.read_json("ticket_data/consolidated.json")
    contagem_registro = df['id'].count()
    print(f"Contagem de linhas: {contagem_registro}")

mostrar_datas_extremas("ticket_data/consolidated.json")


Data mais antiga: 2025-03-12
Data mais recente: 2025-09-07
Contagem de linhas: 814


In [3]:
df_teste = pd.read_json("ticket_data/consolidated.json")
display(df_teste.sort_values(by='date', ascending= False))

,id,module,date,event.id,event.name,event.venue.name,payment.method,payment.method.brand,payment.method.installments,delivery.method,...,ticket.transfer.receiver.gender,ticket.transfer.receiver.birthDate,ticket.transfer.receiver.address.district,ticket.transfer.receiver.address.city,ticket.transfer.receiver.address.state,ticket.entranceDate,ticket.status,virtualBoxOffice,geolocation,status
808,10617687,mobile,2025-09-07 22:34:52+00:00,30617,Camarote Essepê 2026 Desfile Grupo Especial co...,Sambódromo do Anhembi,credit-card,Master,6,virtual,...,None,None,None,None,None,NaN,active,False,{'ip': '2001:12b4:628:6400:c9df:30fa:2f0b:1dd6...,paid
811,10617687,mobile,2025-09-07 22:34:52+00:00,30617,Camarote Essepê 2026 Desfile Grupo Especial co...,Sambódromo do Anhembi,credit-card,Master,6,virtual,...,None,None,None,None,None,NaN,active,False,{'ip': '2001:12b4:628:6400:c9df:30fa:2f0b:1dd6...,paid
810,10617687,mobile,2025-09-07 22:34:52+00:00,30617,Camarote Essepê 2026 Desfile Grupo Especial co...,Sambódromo do Anhembi,credit-card,Master,6,virtual,...,None,None,None,None,None,NaN,active,False,{'ip': '2001:12b4:628:6400:c9df:30fa:2f0b:1dd6...,paid
809,10617687,mobile,2025-09-07 22:34:52+00:00,30617,Camarote Essepê 2026 Desfile Grupo Especial co...,Sambódromo do Anhembi,credit-card,Master,6,virtual,...,None,None,None,None,None,NaN,active,False,{'ip': '2001:12b4:628:6400:c9df:30fa:2f0b:1dd6...,paid
797,10617659,site,2025-09-07 22:23:09+00:00,30617,Camarote Essepê 2026 Desfile Grupo Especial co...,Sambódromo do Anhembi,credit-card,Master,6,virtual,...,None,None,None,None,None,NaN,active,False,"{'ip': '191.19.150.73', 'country': 'BR', 'regi...",paid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,9638934,app,2025-03-12 16:14:19+00:00,30617,Camarote Essepê 2026 Desfile Grupo Especial co...,Sambódromo do Anhembi,credit-card,Master,6,virtual,...,None,None,None,None,None,NaN,active,False,"{'ip': '187.75.108.139', 'country': 'BR', 'reg...",paid
241,9638934,app,2025-03-12 16:14:19+00:00,30617,Camarote Essepê 2026 Desfile Grupo Especial co...,Sambódromo do Anhembi,credit-card,Master,6,virtual,...,None,None,None,None,None,NaN,active,False,"{'ip': '187.75.108.139', 'country': 'BR', 'reg...",paid
0,9638934,app,2025-03-12 16:14:19+00:00,30617,Camarote Essepê 2026 Desfile Grupo Especial co...,Sambódromo do Anhembi,credit-card,Master,6,virtual,...,None,None,None,None,None,NaN,active,False,"{'ip': '187.75.108.139', 'country': 'BR', 'reg...",paid
72,9638914,mobile,2025-03-12 16:12:16+00:00,30617,Camarote Essepê 2026 Desfile Grupo Especial co...,Sambódromo do Anhembi,credit-card,Visa,6,virtual,...,None,None,None,None,None,NaN,active,False,"{'ip': '191.183.40.36', 'country': 'BR', 'regi...",paid
